In [119]:
import import_ipynb
import lr3_x as X
import lr3_y as Y

In [120]:
import pandas as pd
import numpy as np
import math

In [121]:
def localize_space(space):
    return "[{};{})".format(space[0], space[1])

In [122]:
def _value_in_space(value, space):
    return space[0] <= value and value < space[1] 

_source = pd.read_csv('dataset.csv', header=None)
cor = pd.DataFrame()

#cor[0] = [localize_space(X.spaces[i]) for i in range(len(X.spaces))]

for _y, space_y in enumerate(Y.spaces):
    _col = []
    for space_x in X.spaces:
        _count = 0
        for row in _source.iterrows():
            if _value_in_space(row[1][0], space_x) and _value_in_space(row[1][1], space_y):
                _count += 1
        _col.append(_count)
    cor[_y] = _col

cor


,0,1,2,3,4,5,6,7
0,2,0,0,0,0,0,0,0
1,5,4,1,0,0,0,0,0
2,1,3,6,0,0,0,0,0
3,0,3,2,6,1,1,0,0
4,0,0,2,19,11,4,0,0
5,0,0,0,1,4,12,2,0
6,0,0,0,0,1,4,4,4
7,0,0,0,0,0,3,3,5


In [123]:
_rowsIndex = pd.Index([*[localize_space(i) for i in X.spaces], "N"], name="X")
_colsIndex = pd.Index([*[localize_space(i) for i in Y.spaces], "N"], name="Y")
cor1 = cor.copy()
cor1[len(cor1)] = [cor1.iloc[i,:].sum() for i in range(len(X.spaces))]
_sums = [cor1.iloc[:,i].sum() for i in range(len(Y.spaces) + 1)]
cor1 = pd.concat([cor1, pd.DataFrame([_sums])])
cor1.index = _rowsIndex
cor1.columns = _colsIndex
cor1

Y,[84.3;95.3),[95.3;106.3),[106.3;117.3),[117.3;128.3),[128.3;139.3),[139.3;150.3),[150.3;161.3),[161.3;172.3),N
X,,,,,,,,,
[321;350),2,0,0,0,0,0,0,0,2
[350;379),5,4,1,0,0,0,0,0,10
[379;408),1,3,6,0,0,0,0,0,10
[408;437),0,3,2,6,1,1,0,0,13
[437;466),0,0,2,19,11,4,0,0,36
[466;495),0,0,0,1,4,12,2,0,19
[495;524),0,0,0,0,1,4,4,4,13
[524;553),0,0,0,0,0,3,3,5,11
N,8,10,11,26,17,24,9,9,114


In [124]:
import lr2_x
import lr2_y

v_x = np.array(list(map(int, np.round(lr2_x.df["Условные варианты"]))))
v_y = np.array(list(map(int, np.round(lr2_y.df["Условные варианты"]))))

_rowsIndex = pd.Index([*v_x, "n_u"], name="u")
_colsIndex = pd.Index([*v_y, "n_v"], name="v")
cor2 = cor1.copy()
cor2.index = _rowsIndex
cor2.columns = _colsIndex
cor2

v,-4,-3,-2,-1,0,1,2,3,n_v
u,,,,,,,,,
-4,2,0,0,0,0,0,0,0,2
-3,5,4,1,0,0,0,0,0,10
-2,1,3,6,0,0,0,0,0,10
-1,0,3,2,6,1,1,0,0,13
0,0,0,2,19,11,4,0,0,36
1,0,0,0,1,4,12,2,0,19
2,0,0,0,0,1,4,4,4,13
3,0,0,0,0,0,3,3,5,11
n_u,8,10,11,26,17,24,9,9,114


In [125]:
cor3 = cor2.copy()

for i in range(len(v_x)):
    cor3.iloc[i,:-1] *= v_x[i]

for i in range(len(v_y)):
    cor3.iloc[:-1,i] *= v_y[i]

cor3.iloc[:,-1] = cor3.iloc[:,:-1].sum(axis=1)
cor3.iloc[-1,:] = cor3.iloc[:-1,:].sum(axis=0)
cor3

v,-4,-3,-2,-1,0,1,2,3,n_v
u,,,,,,,,,
-4,32,0,0,0,0,0,0,0,32
-3,60,36,6,0,0,0,0,0,102
-2,8,18,24,0,0,0,0,0,50
-1,0,9,4,6,0,-1,0,0,18
0,0,0,0,0,0,0,0,0,0
1,0,0,0,-1,0,12,4,0,15
2,0,0,0,0,0,8,16,24,48
3,0,0,0,0,0,9,18,45,72
n_u,100,63,34,5,0,28,38,69,337


## Вычисление выборочного коэффициента коррелации при помощи условных вариант

In [126]:
_N = lr2_x.N 
_sum = cor3["n_v"]["n_u"]

_counts_x = np.array(list(cor2.iloc[:-1,-1]))
_counts_y = np.array(list(cor2.iloc[-1,:-1]))

_x_mean = sum(_counts_x * v_x) / _N
_y_mean = sum(_counts_y * v_y) / _N

_x_s = np.sqrt((sum(_counts_x * v_x**2) / _N - _x_mean**2))
_y_s = np.sqrt((sum(_counts_y * v_y**2) / _N - _y_mean**2))

r_variants = (_sum - _N * _x_mean * _y_mean) / (_N * _x_s * _y_s)
r_variants 

0.8986484318945402

## Вычисление выборочного коэффициента коррелации стандартным способом

In [129]:
def space_middle(space):
    return (space[1] - space[0]) / 2 + space[0]

_middles_x = list(map(space_middle, X.spaces))
_middles_y = list(map(space_middle, Y.spaces))
cor4 = cor2.copy()

for i in range(len(v_x)):
    cor4.iloc[i,:-1] *= _middles_x[i]

for i in range(len(v_y)):
    cor4.iloc[:-1,i] *= _middles_y[i]

cor4.iloc[:,-1] = cor4.iloc[:,:-1].sum(axis=1)
cor4.iloc[-1,:] = cor4.iloc[:-1,:].sum(axis=0)

_rowsIndex = pd.Index([*_middles_x, "n_u"], name="u")
_colsIndex = pd.Index([*_middles_y, "n_v"], name="v")
cor4.index = _rowsIndex
cor4.columns = _colsIndex

In [130]:
_N = lr2_x.N 
_sum = cor4["n_v"]["n_u"]
r_standard = (_sum - _N * lr2_x.x_middle * lr2_y.x_middle) / (_N * math.sqrt(lr2_x.S2) * math.sqrt(lr2_y.S2))
r_standard

0.8907655509130148